In [ ]:
!pip install transformers -U -q


     |████████████████████████████████| 4.0 MB 5.4 MB/s 
     |████████████████████████████████| 77 kB 5.7 MB/s 
     |████████████████████████████████| 6.6 MB 31.1 MB/s 
     |████████████████████████████████| 596 kB 28.2 MB/s 
     |████████████████████████████████| 880 kB 46.3 MB/s 


In [ ]:
! pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.2 MB/s 


In [ ]:
!pip freeze | grep transformers

transformers==4.18.0


#Importing Libraries

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from sklearn.model_selection import train_test_split
import pandas as pd
import torch

#Google Authentication

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

TIMEOUT: ignored

#Moving data and labels to GPU if available

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


#Training the model on our dataset

##Model

In [ ]:
model1 = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
#tokenizer1 = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX", tgt_lang="hi_IN")
model1 = model1.to(device)

##Tokenizer

In [ ]:
tokenizer1 = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX", tgt_lang="bn_IN")
#tokenizer1 = tokenizer1.to(device)

Downloading:   0%|          | 0.00/528 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/717 [00:00<?, ?B/s]

##Training

In [ ]:
#for i in range(len(trainData)):
file_english = "/content/drive/MyDrive/dataset_for_nlu_project/en-bn/en-bn/train.en"
file_bengali = "/content/drive/MyDrive/dataset_for_nlu_project/en-bn/en-bn/train.bn"

count=0
with open(file_english,"r") as f1, open(file_bengali,"r") as f2:
  for x, y in zip(f1, f2):
     english_text = x.rstrip()
     bengali_text = y.rstrip()
     count=count+1
     model_inputs1 = tokenizer1(english_text, return_tensors="pt").to(device) #source text
     #model_inputs1 = model_inputs1.to(device)
     with tokenizer1.as_target_tokenizer():
        labels1 = tokenizer1(bengali_text, return_tensors="pt").input_ids
        labels1 = labels1.to(device)
     model1(**model_inputs1, labels=labels1)  # forward pass
     if(count%5000 ==0):
        print(count)
     if count==100000:
        print("Training done")
        break      

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
Training done


#Saving the model

In [ ]:
model1.save_pretrained('/content/drive/MyDrive/NLU_project')

#Load the model

In [ ]:
model_pretrained= MBartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/NLU_project')

#Model Architecture

In [ ]:
print(model_pretrained)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,

#Evaluation and Inferencing

#Bleu Score util

In [ ]:
import  nltk.translate.bleu_score as bleu
def BLEU_calculation(test,predicted):
  reference_translation=test.split()
  candidate_translation_1=predicted.split()
  return bleu.sentence_bleu(reference_translation, candidate_translation_1)

#Rouge Score utils

In [ ]:
import itertools
#Split Sentence into Words
def sentence_split_into_words(sent):
  return list(itertools.chain(*[_.split(" ") for _ in sent]))

#Used to  n-grams for multiple sentences
def get_word_ngrams(n, sent_s):

  assert len(sent_s) > 0
  assert n > 0
  w = sentence_split_into_words(sent_s)
  return get_ngrams(n, w)


#Calcualtes n-grams where n: which n-grams to calculate
def get_ngrams(n, text):
  ngram_set = set()
  text_len = len(text)
  max_index_ngram_start = text_len - n
  for k in range(max_index_ngram_start + 1):
    ngram_set.add(tuple(text[k:k + n]))
  return ngram_set

#Calculate ROUGE Score of Refrence and Evaluated Sentences,  n: Size of ngram
def rouge_n(reference_sentences, evaluated_sentences, n=2):
  if len(evaluated_sentences) <= 0 or len(reference_sentences) <= 0:
    raise ValueError("No word in the sentence")

  eval_ngrams = get_word_ngrams(n, evaluated_sentences)
  ref_ngrams = get_word_ngrams(n, reference_sentences)
  reference_count = len(ref_ngrams)
  eval_count = len(eval_ngrams)

  # Gets the overlapping ngrams between evaluated and reference
  overlap_count = len(eval_ngrams.intersection(ref_ngrams))

  if eval_count == 0:
    precision = 0.0
  else:
    precision = overlap_count / eval_count

  if reference_count == 0:
    recall = 0.0
  else:
    recall = overlap_count / reference_count

  f1_score = 2.0 * ((precision * recall) / (precision + recall + 1e-8))

  return recall

#Inferencing

In [ ]:
#for i in range(len(trainData)):
file_english = "/content/drive/MyDrive/dataset_for_nlu_project/en-bn/en-bn/train.en"
file_bengali = "/content/drive/MyDrive/dataset_for_nlu_project/en-bn/en-bn/train.bn"
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")

#ms = round(meteor_score([word_tokenize('John loves Mary')],word_tokenize('John loves Mary')), 4)
rs_score=0
score = 0
count=0
with open(file_english,"r") as f1, open(file_bengali,"r") as f2:
  for x, y in zip(f1, f2):
     count=count+1
     if count>100000:
            english_text = x.rstrip()
            bengali_text = y.rstrip()
            model1_inputs = tokenizer(english_text, return_tensors="pt")
            model1_inputs = model1_inputs.to(device)
            generated_tokens1 = model_pretrained.generate(**model1_inputs,forced_bos_token_id=tokenizer.lang_code_to_id["bn_IN"])
            translation = tokenizer.batch_decode(generated_tokens1, skip_special_tokens=True)
            print(english_text)
            print(bengali_text)
            print(translation)

            #bleu calculation
            bleu_Score = BLEU_calculation(bengali_text,translation[0])
            score=score+bleu_Score
            print("bleu score is ",bleu_Score,sep=" ")

            rs = rouge_n(bengali_text,translation[0])
            print("rouge score is ", rs,sep=" ")
            rs_score=rs_score+rs

            print("********************************************************")
            if count == 100200:
              print("Inference complete")
              
              
              avg_bleu=bleu_Score/200.0
              print("Avg bleu score ",avg_bleu)

              avg_meteor=rs_score/200.0
              print("Avg rouge score ",avg_meteor)

              break
            
                      


ICC Test Team of the Year: Mayank Agarwal, Tom Latham, Marnus Labuschagne, Virat Kohli (captain), Steve Smith, Ben Stokes, BJ Watling (wicketkeeper), Pat Cummins, Mitchell Starc, Neil Wagner, Nathan Lyon.
সেরা টেস্ট দল: মায়াঙ্ক আগরওয়াল, টম লাথাম, মার্নাস লাবুশানে, বিরাট কোহালি (অধিনায়ক), স্টিভ স্মিথ, বেন স্টোকস, বিজে ওয়াটলিং (উইকেটকিপার), প্যাট কামিন্স, মিচেল স্টার্ক, নিল ওয়াগনার, নেথান লায়ন।
['এ বছরের আই সি বড় পরীক ্ ষাটিমটিঃ মায়ান ্ ক আগার ্ वाल, টম ল ্ যাথাম, মা ্ রনাস লাবস ্ চিয়ান, ভিরাট কোলি (கேপটেন), স ্ টিভ স ্ মিথ, ব ্ যান স ্ টোক ্ স, বে জি ওয়াটলিং (বিচ ্ ছক), পেট ক ্ যামেন ্ স, মিশ ্ চল স ্ টারক, নেল ওয়াগ ্ নার, নাথান লিয ় ার ।']
bleu score is  3.739765864071326e-155
rouge score is  0.5483870967741935
********************************************************


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Magway District is a district of the Magway Region (formerly Magway Division) in central Myanmar.
মগওয়ে জেলা () হল কেন্দ্রীয় মিয়ানমারের মগওয়ে অঞ্চলের (সাবেক মগওয়ে বিভাগ) একটি জেলা। শহরাঞ্চল. জেলাতে নিম্নলিখিত শহরাঞ্চল রয়েছে:
['ম ্ যাঙ ্ গুই ক ্ ষেত ্ র হচ ্ ছে ম ্ যাঙ ্ গুই প ্ রদেশের একটি ক ্ ষেত ্ র (সম ্ পুর ্ ন ম ্ যাঙ ্ গুই প ্ রদেশ) কেন ্ দ ্ রস ্ থলে ।']
bleu score is  1.8938679245917267e-78
rouge score is  0.25842696629213485
********************************************************
How do you prepare yourself for a role?
চরিত্রের জন্য কীভাবে নিজেকে প্রস্তুত করেন?
['আপনি কিভাবে নিজের জন ্ য একটা ভূমিকা নেবার জন ্ য প ্ রস ্ তুত হন?']
bleu score is  5.998109029936297e-155
rouge score is  0.5526315789473685
********************************************************
Peter's attitude was wrong.
পিতরের মনোভাব ভুল ছিল।
['পিটারের আচরণ ভুল ছিল ।']
bleu score is  1.2183324802375697e-231
rouge score is  0.5454545454545454
********************************************************


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In the United States only the federal government and state governments are sovereign.
যুক্তরাষ্ট্রে কেন্দ্রীয় সরকার ও রাজ্য সরকারগুলোর সার্বভৌমত্ব রয়েছে।
['যুক ্ তরাষ ্ ট ্ রে কেবলমাত ্ র ফেডারেল আর স ্ টেট এর সরকারই স ্ বাধীন ।']
bleu score is  2.7662422404204523e-78
rouge score is  0.30357142857142855
********************************************************
He had made several allegations.
বেশ কয়েকটি দাবি জানিয়েছেন তিনি।
['সে ভিন ্ ন ভিন ্ ন ধরণের আপত ্ তি তুলে ধরেছিল ।']
bleu score is  1.2395288183339461e-231
rouge score is  0.25
********************************************************
"Taking to Twitter Yuvraj wrote, ""Happy Birthday to the undisputed Dada of Indian Cricket."
যুবরাজ সিংহ ট্যুইটারে লিখেছেন, 'ভারতীয় ক্রিকেটের তর্কাতীত দাদাকে জন্মদিনের শুভেচ্ছা।
['‘ ট ্ রিউটারে করে দেখুন, এক ্ ষেত ্ রে জ ্ ঞানযোগ ্ য ইন ্ ডিয়ান ক ্ রিke ্ ট ডা ডা ডাকে শুভ সন ্ বর ্ গ । ’']
bleu score is  5.436761716113821e-155
rouge score is  0.2894736842105263
*************************************

#Given one line,inference it

In [ ]:
model_pretrained= MBartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/NLU_project')
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")

In [ ]:
article_en = "I love my India"
model_inputs = tokenizer(article_en, return_tensors="pt")
generated_tokens = model_pretrained.generate(**model_inputs,forced_bos_token_id=tokenizer.lang_code_to_id["bn_IN"])
translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(translation)


'I love my India'